In [1]:
import pandas as pd
from collections import Counter
import numpy as np

df = pd.read_csv("fake_data.csv", encoding_errors="ignore", on_bad_lines='skip', sep=",",)
# df = pd.read_csv("BPI2016_Clicks_Logged_In.csv", encoding_errors="ignore", on_bad_lines='skip', sep=";",
#                 usecols=['CustomerID', 'AgeCategory', 'Gender', 'Office_U', 'Office_W',
#        'SessionID', 'IPID', 'TIMESTAMP', 'VHOST', 'URL_FILE', 'PAGE_NAME',
#        'REF_URL_category', 'page_load_error', 'page_action_detail', 'tip',
#        'service_detail', 'xps_info'])
# df.rename(columns={'URL_FILE':'Activity'}, inplace=True)

In [2]:
df["TIMESTAMP"] = pd.to_datetime(df["TIMESTAMP"], infer_datetime_format=True)
df = df.sort_values(["SessionID", "TIMESTAMP"])

In [3]:
#Create consecutive column with the consecutive activity and count how many time each pair occurs
df["Consecutive_1"] = df.groupby("SessionID")["Activity"].shift(periods=-1)

#Count all the pairs and safe in new DataFrame
df_for_pivot_1 = pd.DataFrame(Counter(list(zip(df['Activity'], df['Consecutive_1'],))), index=["value"]).T.reset_index()

df_for_pivot_1.fillna("end_session", inplace=True)
df_for_pivot_1

,level_0,level_1,value
0,Start_application,Input_info,2
1,Input_info,Send_application,2
2,Send_application,Accept_offer,3
3,Accept_offer,end_session,4
4,Input_info,end_session,1


In [4]:
def make_pivot(df, index_names, column_names):
    """
    Functions that takes in a DataFrame and returns a pivot table with all the chances
    
    Create pivot table where chances are calculated that each row is succeded by activity that is represented in the column
    """
    df_chances = df.pivot_table(index=index_names, columns=column_names, values='value')
    
    #Calculate what the probabilities are by summing the row and dividing all the values in the row by total sum of the row
    df_chances["total_row_count"] = df_chances.sum(axis=1)
    df_chances = df_chances.div(df_chances["total_row_count"], axis=0)
    
    df_chances.fillna(0, inplace=True)
    df_chances.drop("total_row_count", axis=1, inplace=True)

    return df_chances

In [5]:
df_chances_1 = make_pivot(df_for_pivot_1, "level_0", "level_1")

#Calculate the probability of each activity that it is the first activity performed in the session
df_chances_1["start_session_chance"] = df.groupby("SessionID").nth(0)["Activity"].value_counts() / df.groupby("SessionID").nth(0)["Activity"].value_counts().sum()
df_chances_1.fillna(0, inplace=True)

In [6]:
df_chances_total = df_chances_1 

#Split the start probabilities from the normal DataFrame
start_chances = df_chances_total["start_session_chance"][df_chances_total["start_session_chance"] > 0]

#Create df with all chances except the starting chance.
df_chances_rest = df_chances_total.iloc[:, :-1]

In [7]:
df_chances_rest

level_1,Accept_offer,Input_info,Send_application,end_session
level_0,,,,
Accept_offer,0.0,0.0,0.000000,1.000000
Input_info,0.0,0.0,0.666667,0.333333
Send_application,1.0,0.0,0.000000,0.000000
Start_application,0.0,1.0,0.000000,0.000000


Try to find paths of length 3.
A - B - C
Sort on chance. 
So chance a to b * b to c 

In [8]:
def find_paths(_df:pd.DataFrame,len:int = 3):
  df = _df.copy()
  col = df.columns.values
  row  = df.index.values
  s  = row.size
  res = []
  while s > 0:
    current_row = df.iloc[row.size - s]
    i = 0
    while i < current_row.size:
      current_col = df.columns[i]
      chance = current_row.iloc[i]
      if chance > 0:
        a = current_row.name
        b = current_col
        
        if (df.index == b).any() :
          _r = df.loc[b]
          index = 0
          for r in _r:
            if r > 0:
              res.append([a,b,col.item(index),r*chance,chance,r])  
            index += 1        
            
      i += 1
    s -= 1

  return res


paths = find_paths(df_chances_rest)

In [9]:
df_paths = pd.DataFrame(paths,columns=['a','b','c','a->c','a->b', 'b->c'])

In [10]:
df_paths.sort_values(by= ["a->c"],ascending=False,inplace=True,ignore_index=True)
df_paths

,a,b,c,a->c,a->b,b->c
0,Send_application,Accept_offer,end_session,1.000000,1.000000,1.000000
1,Input_info,Send_application,Accept_offer,0.666667,0.666667,1.000000
2,Start_application,Input_info,Send_application,0.666667,1.000000,0.666667
3,Start_application,Input_info,end_session,0.333333,1.000000,0.333333


In [15]:
df_chances_rest

level_1,Accept_offer,Input_info,Send_application,end_session
level_0,,,,
Accept_offer,0.0,0.0,0.000000,1.000000
Input_info,0.0,0.0,0.666667,0.333333
Send_application,1.0,0.0,0.000000,0.000000
Start_application,0.0,1.0,0.000000,0.000000


In [78]:
def skip_event_all_possibilities(_df:pd.DataFrame,event:str,deviation_rate):
  df = _df.copy()
  print(f"Decrease the probability to reach event: {event} with deviation rate: {deviation_rate}")
  old_probability = df.loc[:,event]
  deviation_rate = deviation_rate / 100
  
  # decrease the probability
  for i,prob in old_probability.items():
    if prob > 0 and prob - deviation_rate > 0:
      print(f"from event: {i} to: {event}")
      decrease = prob * (1-deviation_rate)
      
      print(f"old value: {prob}")
      print(f"decrease: {prob - decrease}")
      print(f"new value: {decrease}")
      
      df.loc[i,event] -= prob - decrease
      
      for _i,_prob in df.loc[i,:].items():
        df.loc[i,_i] += (prob - decrease) * df.loc[event,_i]
      
      
    elif prob > 0 and prob - deviation_rate < 0:
      prob = 0
  
  return df
       

skip_event_all_possibilities(df_chances_rest,"Send_application",10)


  

Decrease the probability to reach event: Send_application with deviation rate: 10
from event: Input_info to: Send_application
old value: 0.6666666666666666
decrease: 0.06666666666666665
new value: 0.6


level_1,Accept_offer,Input_info,Send_application,end_session
level_0,,,,
Accept_offer,0.000000,0.0,0.0,1.000000
Input_info,0.066667,0.0,0.6,0.333333
Send_application,1.000000,0.0,0.0,0.000000
Start_application,0.000000,1.0,0.0,0.000000


In [12]:
def markov_chain(amount, start_chances, df_dict):
    lijst_aantal_gegenereerde = []

    #Loop through the df with all the chances and based on this a new session is generated
    aanvragen_ww_list = []
    while len(aanvragen_ww_list) <= amount:
        activity_list = []
        activity = np.random.choice(a = np.array(start_chances.index), size = 1, p = np.array(start_chances.values))
        activity_list.append(activity[0])

        while activity != "end_session":
            row_activity = df_dict[activity[0]]
            activity = np.random.choice(a = list(row_activity.keys()), size = 1, p = list(row_activity.values()))
            activity_list.append(activity[0])

        if "/portal/page/portal/home/diensten/aanvragen-ww" in activity_list:
            aanvragen_ww_list.append(activity_list)

        #print how many sessions are already generated by the Markov chain. Only print every 250 generated sessions
        if len(aanvragen_ww_list) % 250 == 0 and len(aanvragen_ww_list) not in lijst_aantal_gegenereerde:
            print(len(aanvragen_ww_list))
            lijst_aantal_gegenereerde.append(len(aanvragen_ww_list))
    
    return aanvragen_ww_list

markov_result = markov_chain(2000, start_chances, df_dict)
# markov_result

NameError: name 'df_dict' is not defined

In [ ]:
# import matplotlib.pyplot as plt
# import math


# def round_decimals_down(number:float, decimals:int=1):
#     """
#     Returns a value rounded down to a specific number of decimal places.
#     """
#     if not isinstance(decimals, int):
#         raise TypeError("decimal places must be an integer")
#     elif decimals < 0:
#         raise ValueError("decimal places has to be 0 or more")
#     elif decimals == 0:
#         return math.floor(number)

#     factor = 10 ** decimals
#     return math.floor(number * factor) / factor


# data = df_test.head(101)["a->c"]
# ranges = [1,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.1,0.0]
# # count = [0,0,0,0,0,0,0,0,0,0]

# new_df = pd.DataFrame(columns=ranges,index=["a->c"])
# new_df.loc['a->c'] = pd.Series({1:0,0.9:0,0.8:0,0.7:0,0.6:0,0.5:0,0.4:0,0.3:0,0.1:0,0.0:0})

# for d in data:
#   if not new_df[round_decimals_down(d)][0] :
#     new_df[round_decimals_down(d)][0] = 1
#   else:
#     new_df[round_decimals_down(d)][0] += 1
    
# # new_df.plot.barh()


In [ ]:
# def create_test_df():
#   d = {'B': [1,0, 0.5,0], 'C': [0,0.5,0,0.2],'D':[0,0.5,0.5,0],'E':[0,0,0,0.8]}
#   df = pd.DataFrame(data=d)
#   df.index = ['A','B','C','D']
#   return df

# create_test_df()  

,B,C,D,E
A,1.0,0.0,0.0,0.0
B,0.0,0.5,0.5,0.0
C,0.5,0.0,0.5,0.0
D,0.0,0.2,0.0,0.8
